In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [13]:
merged_data = pd.read_csv('merged_data_with_predictions.csv')

In [14]:
# Assuming merged_data contains the merged features and labels
X = pd.get_dummies(merged_data[['url_len', 'domain', '@', '?', '-', '=', '.', '#', '%', '+', '$', '!', '*', ',', '//', 'abnormal_url', 'https', 'digits', 'letters', 'Shortining_Service', 'having_ip_address']])
y = merged_data['label_email']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
y_train.value_counts()

label_email
0    561
1    249
Name: count, dtype: int64

In [17]:
models = {
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC()
}

In [18]:
# Evaluate models using cross-validation
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    print(f'{name} Accuracy: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})')

Random Forest Accuracy: 0.9398 (+/- 0.0145)


c:\Users\toufic.f\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\toufic.f\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.or

Logistic Regression Accuracy: 0.9763 (+/- 0.0171)
SVM Accuracy: 0.7009 (+/- 0.0034)


In [19]:
# Choosen Logistic Regression based on cross-validation results
best_model = LogisticRegression()  
best_model.fit(X_train, y_train)

# Make predictions and evaluate the best model
preds = best_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print('Best Model Accuracy:', acc)

# Print classification report
print('Classification report:')
print(classification_report(y_test, preds))

# Print confusion matrix
print('Confusion matrix:')
print(confusion_matrix(y_test, preds))

Best Model Accuracy: 0.9802955665024631
Classification report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       149
           1       0.96      0.96      0.96        54

    accuracy                           0.98       203
   macro avg       0.97      0.97      0.97       203
weighted avg       0.98      0.98      0.98       203

Confusion matrix:
[[147   2]
 [  2  52]]


c:\Users\toufic.f\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
